### Setup

First of all, we authenticate a Google Drive client to download the dataset we will be processing in this Colab.

**Make sure to follow the interactive instructions.**

In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:
id='1EoolSK32_U74I4FeLox88iuUB_SUUYsI'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('web-Stanford.txt')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Data Loading

For this Colab we will be using [NetworkX](https://networkx.github.io), a Python package for the creation, manipulation, and study of the structure, dynamics, and functions of complex networks.

The dataset we will analyze is a snapshot of the Web Graph centered around [stanford.edu](https://stanford.edu), collected in 2002. Nodes represent pages from Stanford University (stanford.edu) and directed edges represent hyperlinks between them. [[More Info]](http://snap.stanford.edu/data/web-Stanford.html)

In [4]:
import networkx as nx

G = nx.read_edgelist('web-Stanford.txt', create_using=nx.DiGraph)

In [5]:
print(nx.info(G))

DiGraph with 281903 nodes and 2312497 edges


### Your Task

To begin with, let's simplify our analysis by ignoring the dangling nodes and the disconnected components in the original graph.

Use NetworkX to identify the **largest** weakly connected component in the ```G``` graph.  From now on, use this connected component for all the following tasks.

In [6]:
# YOUR CODE HERE
print('nodes:',G.number_of_nodes())
print('edges:',G.number_of_edges())

lwcc = G.subgraph(max(nx.weakly_connected_components(G), key=len))
print('largest weakly connected component:',nx.info(lwcc))

nodes: 281903
edges: 2312497
largest weakly connected component: DiGraph with 255265 nodes and 2234572 edges


Compute the PageRank vector, using the default parameters in NetworkX: [https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html#networkx.algorithms.link_analysis.pagerank_alg.pageranky](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html#networkx.algorithms.link_analysis.pagerank_alg.pagerank)

In [9]:
# YOUR CODE HERE
pr = nx.pagerank(lwcc)

# Top 10 pages
count = 0
for i in sorted(pr.items(), key=lambda item: item[1], reverse=True):
    print(i)
    count += 1
    if count > 10:
        break


('89073', 0.011051585882434985)
('226411', 0.010694113250567905)
('241454', 0.009829260884719991)
('134832', 0.00650923773721211)
('69358', 0.003753708143672675)
('67756', 0.003543473943866138)
('105607', 0.0032305919516859047)
('225872', 0.0031736850016296342)
('234704', 0.0031708863624340614)
('186750', 0.00314345200380852)
('231363', 0.003061480040107788)


In 1999, Barabási and Albert proposed an elegant mathematical model which can generate graphs with topological properties similar to the Web Graph (also called Scale-free Networks).

If you complete the steps below, you should obtain some empirical evidence that the Random Graph model is inferior compared to the Barabási–Albert model when it comes to generating a graph resembling the World Wide Web!

As such, we will use two different graph generator methods, and then we will test how well they approximate the Web Graph structure by means of comparing the respective PageRank vectors. [[NetworkX Graph generators]](https://networkx.github.io/documentation/stable/reference/generators.html#)

Using for both methods ```seed = 1```, generate:


1.   a random graph (with the fast method), setting ```n``` equal to the number of nodes in the original connected component, and ```p = 0.00008```
2.   a Barabasi-Albert graph (with the standard method), setting ```n``` equal to the number of nodes in the original connected component, and finding the right ***integer*** value for ```m``` such as the resulting number of edges **approximates by excess** the number of edges in the original connected component

and compute the PageRank vectors for both graphs.


In [10]:
# YOUR CODE HERE

n = lwcc.number_of_nodes()
#1 Random Graph
p =0.00008

RG = nx.fast_gnp_random_graph(n, p, seed=1, directed=False)

#2 Barabasi-Alber Graph
# m must be <= n

BAG = nx.barabasi_albert_graph(n, n-1, seed=1, initial_graph=None)

In [11]:
pr_RG = nx.pagerank(RG)
# Top 10 pages
count = 0
for i in sorted(pr_RG.items(), key=lambda item: item[1], reverse=True):
    print(i)
    count += 1
    if count > 10:
        break

(150787, 7.458986619588909e-06)
(14579, 7.3974735409810506e-06)
(193475, 7.368176905091742e-06)
(141505, 7.348699650919032e-06)
(144307, 7.323021888645532e-06)
(184745, 7.313541287755495e-06)
(129032, 7.299830669613334e-06)
(70124, 7.268730168861052e-06)
(239691, 7.228833381965614e-06)
(127392, 7.197456973720354e-06)
(198700, 7.170042370818152e-06)


In [12]:
pr_BAG = nx.pagerank(BAG)
# Top 10 pages
count = 0
for i in sorted(pr_BAG.items(), key=lambda item: item[1], reverse=True):
    print(i)
    count += 1
    if count > 10:
        break

(0, 0.5150105145690855)
(1, 1.89995254100808e-06)
(2, 1.89995254100808e-06)
(3, 1.89995254100808e-06)
(4, 1.89995254100808e-06)
(5, 1.89995254100808e-06)
(6, 1.89995254100808e-06)
(7, 1.89995254100808e-06)
(8, 1.89995254100808e-06)
(9, 1.89995254100808e-06)
(10, 1.89995254100808e-06)


Compare the PageRank vectors obtained on the generated graphs with the PageRank vector you computed on the original connected component.
**Sort** the components of each vector by value, and use cosine similarity as similarity measure. 

Feel free to use any implementation of the cosine similarity available in third-party libraries, or implement your own with ```numpy```.

In [21]:
# YOUR CODE HERE
import numpy as np
from numpy.linalg import norm

# Cosine Similarity is define as Similarity = (A.B) / (||A||.||B||) 

def CosineSimilarity(A,B):
  cosine = np.dot(list(A),list(B))/(norm(list(A))*norm(list(B)))
    
  return cosine
  
print("sim(pr_RG,pr_BAG) = ",CosineSimilarity(pr_RG, pr_BAG))

sim(pr_RG,pr_BAG) =  0.9999999999999998


#### **Submission Intruction:**

#### Click File -> Download -> Download **.ipynb**, and upload the downloaded file to Blackboard.